In [6]:
## - python-docx
# %pip install -qU python-docx

import tiktoken
import chromadb
import os
from openai import OpenAI
from docx import Document
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

encoding_model = tiktoken.encoding_for_model("gpt-4o")

## 임베딩 모델 지정
OPEN_API_KEY = os.getenv('OPENAI_API_KEY')
openai_embedding = OpenAIEmbeddingFunction(
    api_key=OPEN_API_KEY,
    model_name = 'text-embedding-3-large'
)

## 1. 문서 내용 읽기
document = Document('law_1.docx')


## 읽어 온 단락을 하나의 문서로 만들기
full_text =''
for paragraph in document.paragraphs:
    full_text += paragraph.text + "\n"

## 2. 문서 분할(쪼개기)
## %pip install tiktoken
## 토큰화
encoding = encoding_model.encode(full_text)

def split_text(full_text, chunk_size):
    encoder_model = tiktoken.encoding_for_model('gpt-4o')
    total_encoding = encoder_model.encode(full_text)
    total_token_count = len(total_encoding)

    text_list= []
    for i in range(0, total_token_count, chunk_size):
        chunk = total_encoding[i: i+chunk_size]
        decoding= encoding_model.decode(chunk)
        text_list.append(decoding)
    
    return text_list

chunk_list = split_text(full_text, 1500)

## 3. 벡터 데이터베이스에 저장
## %pip install chromadb
## 크로마 클라이언트 생성

chroma_client = chromadb.Client()
collection_name = 'law_collection'

law_collection = chroma_client.get_or_create_collection(
    collection_name, 
    embedding_function=openai_embedding,
    )

## 컬렉션에 설정할 인덱스 생성
id_list = []

for index in range(len(chunk_list)):
    id_list.append(f'{index}')

law_collection.add(documents=chunk_list, ids=id_list)

## 4. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색
query = '전세사기피해자 금융지원에 대해 설명해주세요.'

## 유사도 검색
retrieved_doc = law_collection.query(query_texts=[query], n_results=3)

## 5.질문 -> 답변
## 5.1 
# from openai import OpenAI
# client = OpenAI()

# response = client.responses.create(
#     model="gpt-4o",
#     instructions=f'''
#     - 당신은 전세사기피해 법률 전문가입니다.
#     - [context]를 참고하여사용자의 질문에 답변해주세요.
#     [context]
#     {retrieved_doc['documents'][0]}
#     ''',
#     input=query,
# )

# print(response.output_text)

## 5.2
client = OpenAI()

response = client.responses.create(
    model="gpt-4o",
    input=[
        {
            "role": "developer",
            "content": f'''- 당신은 전세사기피해 법률 전문가입니다.
            - [context]를 참고하여사용자의 질문에 답변해주세요.
             [context]
            {retrieved_doc['documents'][0]}
            '''
        },
        {
            "role": "user",
            "content": query
        }
    ]
)

print(response.output_text)

전세사기피해자에 대한 금융지원은 다음과 같은 내용을 포함하고 있습니다.

1. **자금 융자**: 국가 및 지방자치단체는 전세사기피해자와 임차인의 주거안정을 위해 필요한 자금을 지원할 수 있습니다.

2. **주택도시기금 지원**: 주택임대차보호법에 따른 우선변제를 받지 못한 전세사기피해자에게는 주택도시기금에서 필요한 자금을 융자할 수 있습니다.

이러한 지원은 전세사기피해자들이 경제적 부담을 덜고 주거 안정을 되찾을 수 있도록 돕기 위한 제도적 장치입니다.
